In [1]:
# Run setup_imports to enable importing from src/
%run ../src/setup_imports.py

In [2]:
from src.utils.ibm import get_backend, get_backend_snapshot, run_and_wrap
from src.utils.python_utils import clean_datetimes
from src.utils.circuits import chsh_circuit, phi_plus_meassured, compute_entanglement_correlation
from qiskit_ibm_runtime import SamplerV2 as Sampler
from datetime import datetime
import json
import time
import pandas as pd

/Volumes/X-Wing/casco/Development/introspect-execute-time-drift-experiment/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Parametrize experiment
reps = 1
delay_between_reps = 60 # seconds
max_duration = 10 * 60 * 60  # 10 hours in seconds
introspection_shots = 1024
main_circuit_shots = 1024
entanglement_threshold = 0.4

In [4]:
backend = get_backend()

In [ ]:
# Main experiment loop

results = []
start_time = time.time()

i = 0
while i < reps:
    entry = {}
    
    # --- Backend snapshot; need we do this more than once?---
    entry['backend_snapshot'] = get_backend_snapshot(backend)

    # --- Introspection phase ---
    print(f"{i}. Running introspection circuit")
    entry['introspection'] = run_and_wrap(chsh_circuit(), backend, introspection_shots)
    entry['entanglement_score'] = compute_entanglement_correlation(entry['introspection']["counts"])

    # --- Decision phase ---
    if entry['entanglement_score'] > entanglement_threshold:  
        print(f"{i}. Running main circuit - entanglement score was {entry['entanglement_score']}")
        entry['execution'] = run_and_wrap(phi_plus_meassured(), backend, main_circuit_shots)
    else:
        print(f"{i}. Skipping main circuit - entanglement score was {entry['entanglement_score']}")
    
    results.append(clean_datetimes(entry))
    
    if i < reps:
        time.sleep(delay_between_reps) # wait before next repetition 

# Save results
with open("experiment_results.json", "w") as f:
    json.dump(results, f, indent=2)

# Tabular summary
df = pd.DataFrame([{
    'introspection_created': r.get('introspection', {}).get('timestamps', {}).get('created'),
    'introspection_running': r.get('introspection', {}).get('timestamps', {}).get('running'),
    'introspection_finished': r.get('introspection', {}).get('timestamps', {}).get('finished'),
    'execution_created': r.get('execution', {}).get('timestamps', {}).get('created'),
    'execution_running': r.get('execution', {}).get('timestamps', {}).get('running'),
    'execution_finished': r.get('execution', {}).get('timestamps', {}).get('finished'),    
    'entanglement_score': r.get('entanglement_score'),
} for r in results])
df.to_csv("experiment_summary.csv", index=False)

0. Running introspection circuit


/Volumes/X-Wing/casco/Development/introspect-execute-time-drift-experiment/.venv/lib/python3.9/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:1111: UserWarning: IBM Quantum Platform is moving, and this version will be sunset on July 1. To get started on the new platform, read the migration guide at https://docs.quantum.ibm.com/migration-guides/classic-iqp-to-cloud-iqp
  warnings.warn(warning_message)


0. Running main circuit - entanglement score was 0.505859375
